In [ ]:
%matplotlib inline

import sys
sys.path.append('/home/wt/py_projects/HPE-3d')
sys.path.append('/home/wt/py_projects/HPE-3d/datasets')


import numpy as np
import matplotlib.pyplot as plt
from datasets.h36m import Human36mDataset

In [ ]:
dir_path = '../data/'
data_path = dir_path + 'data_2d_h36m_gt.npz'
data_path_3d = dir_path + 'data_3d_h36m.npz'
keypoints = np.load(data_path, allow_pickle=True)['positions_2d'].item()
dataset = Human36mDataset(data_path_3d,True)
print(dataset._skeleton.num_joints())


In [ ]:
camera = 0
kps = keypoints['S1']['Photo'][camera] 
anim = dataset['S1']['Photo']
jts = dataset['S1']['Photo']['positions']

print(kps.shape)
print(jts.shape)
frame = 800

coordinates_2d = kps[frame]
coordinates_3d = jts[frame] # global
print(coordinates_2d)
print(coordinates_3d)

In [ ]:
joint_pairs = [[0, 1], [1, 2], [2, 3], [0, 4], [4, 5], [5, 6],
               [0, 7], [7, 8], [8, 9], [9, 10], [8, 11], [11, 12],
               [12, 13], [8, 14], [14, 15], [15, 16]]

colors_kps = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0],
              [50, 205, 50], [0, 255, 170], [0, 255, 255], [
                  0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255],
              [170, 0, 255], [255, 0, 255]]


def plot_keypoint(coordinates):
    plt.figure(figsize=(7,7))
    x_max = coordinates_2d[:, 0].max()
    x_min = coordinates_2d[:, 0].min()
    y_max = coordinates_2d[:, 1].max()
    y_min = coordinates_2d[:, 1].min()
    # axes.set_xlim([0,h36m_cameras_intrinsic_params['res_w']])
    # axes.set_ylim([h36m_cameras_intrinsic_params['res_h'],0])
    plt.xlim(x_min-20,x_max+20)
    plt.ylim(y_max+20,y_min-20)
    plt.axis('equal')

    for i in range(coordinates.shape[0]):
        pts = coordinates[i]
        for color_i, jp in zip(colors_kps, joint_pairs):
            color_i = [c / 255 for c in color_i]
            pt_a = coordinates[jp[0]]
            pt_b = coordinates[jp[1]]
            # pt_a_x, pt_a_y, pt_b_x, pt_b_y = int(pt_a[0]), int(
            #     pt_a[1]), int(pt_b[0]), int(pt_b[1])
            pt_a_x, pt_a_y, pt_b_x, pt_b_y = pt_a[0],pt_a[1], pt_b[0], pt_b[1]
            plt.plot((pt_a_x, pt_b_x), (pt_a_y, pt_b_y), color=color_i, lw=2.0)
            plt.plot(pt_a_x,pt_a_y,marker='o',color=color_i)
            plt.plot(pt_b_x,pt_b_y,marker='o',color=color_i)


plot_keypoint(coordinates_2d)


In [ ]:
# from common.camera import *
# cam = anim['cameras'][0]
# print(cam)

# pos_3d = world_to_camera(anim['positions'],R=cam['orientation'],t=cam['translation'])
# print(pos_3d.shape)
# print(pos_3d[:,:1].shape)
from common.camera import *

cam  =anim['cameras'][camera]
pos_3d = world_to_camera(anim['positions'],R=cam['orientation'],t=cam['translation'])
# Remove global offset, but keep trajectory in first position
# 这里是将0号关节置于相机坐标系的中心
# 因此pos_3d[:, 0]本应该全置为0， 但是代码里没有在这里做，而是在训练阶段做
# 即 inputs_3d[:, :, 0] = 0
print(pos_3d[frame])
pos_3d[:, 1:] -= pos_3d[:, :1]
pos_3d[:,0] = 0


print(pos_3d)

In [ ]:
coordinates_3d = pos_3d[frame] # camera space
print(coordinates_3d.shape)
print(coordinates_3d)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ymin = coordinates_3d.min()
ymax = coordinates_3d.max()

ax.set_zlim([ymax,ymin])
# ax.set_xlim([ymin,ymax])
# ax.set_ylim([ymax,ymin])
# ax.autoscale_view()
ax.set_aspect('equal')

for color_i, jp in zip(colors_kps, joint_pairs):
    color_i = [c / 255 for c in color_i]
    pt_a = coordinates_3d[jp[0]]
    pt_b = coordinates_3d[jp[1]]
    # print(pt_a,pt_b)
    pt_a_x, pt_a_y, pt_a_z, pt_b_x, pt_b_y, pt_b_z = pt_a[
        0],   pt_a[1], pt_a[2], pt_b[0], pt_b[1], pt_b[2]
    # print(pt_a_z,pt_b_z)

    ax.plot((pt_a_x, pt_b_x), (pt_a_z, pt_b_z),
            (pt_a_y, pt_b_y), lw=2.0, c=color_i)
    ax.plot(pt_a_x, pt_a_z, pt_a_y, marker='o', color=color_i)
    ax.plot(pt_b_x, pt_b_z, pt_b_y, marker='o', color=color_i)
